In [43]:
import numpy as np 
from transformers import AutoTokenizer
from torch.utils.data import Dataset
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader
import torch

# Convert data to format

- images to temporal images
- text to encodings

In [44]:
X = torch.rand((3,3,50,10))


String conversion

In [45]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")  # or "gpt2", etc.

text = ["rough surface", "sliding contact", "object tilting"]
tokenized = tokenizer(text, padding="max_length", truncation=True, return_tensors="pt")
print(tokenized)
input_ids=tokenized['input_ids']
print(input_ids)  
decoded=tokenizer.decode(input_ids[0].squeeze(), skip_special_tokens=True)
print(decoded)
y=decoded

{'input_ids': tensor([[ 8678,  1774,     1,  ...,     0,     0,     0],
        [13441,   574,     1,  ...,     0,     0,     0],
        [ 3735, 20382,    53,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
tensor([[ 8678,  1774,     1,  ...,     0,     0,     0],
        [13441,   574,     1,  ...,     0,     0,     0],
        [ 3735, 20382,    53,  ...,     0,     0,     0]])
rough surface


In [46]:
def tokenize_captions(captions, max_len=30):
    return tokenizer(
        captions,
        padding="max_length",
        truncation=True,
        max_length=max_len,
        return_tensors="pt"
    )

# Train model to predict text

In [47]:
class TactileDataset(Dataset):
    def __init__(self, images, captions, tokenizer, max_len=30):
        self.images = images
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.captions = captions

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        caption = self.captions[idx]

        tokens = self.tokenizer(caption, max_length=self.max_len,
                                padding="max_length", truncation=True,
                                return_tensors="pt")
        
        return {
            "image": image,
            "input_ids": tokens["input_ids"].squeeze(0),
            "attention_mask": tokens["attention_mask"].squeeze(0)
        }


In [48]:
class TactileCaptioningModel(nn.Module):
    def __init__(self, vocab_size, embed_dim=256, hidden_dim=512, max_len=30):
        super().__init__()
        # Image encoder: ResNet18 without classifier
        resnet = models.resnet18(pretrained=True)
        resnet.fc = nn.Identity()
        self.encoder = resnet

        # Project image features
        self.img_proj = nn.Linear(512, embed_dim)

        # Text decoder: Embedding + Transformer Decoder
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        decoder_layer = nn.TransformerDecoderLayer(d_model=embed_dim, nhead=4)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=2)

        self.output_layer = nn.Linear(embed_dim, vocab_size)

    def forward(self, images, input_ids):
        batch_size = images.size(0)
        image_features = self.encoder(images)  # (B, 512)
        image_features = self.img_proj(image_features)  # (B, E)
        image_features = image_features.unsqueeze(1)  # (B, 1, E)

        tgt = self.embedding(input_ids)  # (B, T, E)
        tgt = tgt.transpose(0, 1)  # (T, B, E)
        memory = image_features.transpose(0, 1)  # (1, B, E)

        out = self.transformer_decoder(tgt, memory)  # (T, B, E)
        out = out.transpose(0, 1)  # (B, T, E)

        return self.output_layer(out)

In [50]:
model = TactileCaptioningModel(vocab_size=tokenizer.vocab_size)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

dataset = TactileDataset(X, y, tokenizer)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

for epoch in range(100):
    model.train()
    total_loss = 0
    for batch in dataloader:
        images = batch["image"].to(device)
        input_ids = batch["input_ids"].to(device)

        outputs = model(images, input_ids[:, :-1])
        loss = loss_fn(outputs.reshape(-1, tokenizer.vocab_size), input_ids[:, 1:].reshape(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch}: Loss = {total_loss / len(dataloader):.4f}")

Epoch 0: Loss = 10.8935
Epoch 1: Loss = 9.8110
Epoch 2: Loss = 8.7889
Epoch 3: Loss = 8.0141
Epoch 4: Loss = 7.2723
Epoch 5: Loss = 6.5863
Epoch 6: Loss = 6.1406
Epoch 7: Loss = 5.6804
Epoch 8: Loss = 5.4007
Epoch 9: Loss = 4.9795
Epoch 10: Loss = 4.8030
Epoch 11: Loss = 4.5864
Epoch 12: Loss = 4.4110
Epoch 13: Loss = 4.1720
Epoch 14: Loss = 4.0395
Epoch 15: Loss = 4.0044
Epoch 16: Loss = 3.8023
Epoch 17: Loss = 3.7459
Epoch 18: Loss = 3.5053
Epoch 19: Loss = 3.4694
Epoch 20: Loss = 3.2419
Epoch 21: Loss = 3.2111
Epoch 22: Loss = 3.0373
Epoch 23: Loss = 2.9363
Epoch 24: Loss = 2.8049
Epoch 25: Loss = 2.6922
Epoch 26: Loss = 2.5326
Epoch 27: Loss = 2.4030
Epoch 28: Loss = 2.2992
Epoch 29: Loss = 2.2733
Epoch 30: Loss = 1.9953
Epoch 31: Loss = 2.0527
Epoch 32: Loss = 1.9550
Epoch 33: Loss = 1.8417
Epoch 34: Loss = 1.6781
Epoch 35: Loss = 1.6407
Epoch 36: Loss = 1.5731
Epoch 37: Loss = 1.6151
Epoch 38: Loss = 1.4555
Epoch 39: Loss = 1.3644
Epoch 40: Loss = 1.4308
Epoch 41: Loss = 1.3380
E

In [51]:
def generate_caption(model, image, tokenizer, max_len=30, device='cpu'):
    model.eval()
    image = image.to(device)

    with torch.no_grad():
        # Step 1: Encode image
        image_features = model.encoder(image)  # (1, 512)
        image_features = model.img_proj(image_features).unsqueeze(1).transpose(0, 1)  # (1, 1, embed_dim)

        # Step 2: Start decoding
        input_ids = torch.tensor([[tokenizer.pad_token_id]], device=device)  # (1, 1)

        for _ in range(max_len):
            tgt = model.embedding(input_ids)  # (1, T, E)
            tgt = tgt.transpose(0, 1)         # (T, 1, E)

            output = model.transformer_decoder(tgt, image_features)  # (T, 1, E)
            last_token_logits = model.output_layer(output[-1])       # (1, vocab_size)
            next_token_id = torch.argmax(last_token_logits, dim=-1)  # (1,)

            # Append to sequence
            input_ids = torch.cat([input_ids, next_token_id.unsqueeze(0)], dim=1)

            # Stop if EOS
            if next_token_id.item() == tokenizer.eos_token_id:
                break

        decoded = tokenizer.decode(input_ids.squeeze(), skip_special_tokens=True)
        return decoded

In [56]:
caption = generate_caption(model, torch.rand(1,3,10,50), tokenizer, max_len=30, device=device)
print("Predicted caption:", caption)

Predicted caption: 


# Use the text with another LLM